# 1. Data Acquisition and Consolidation

In [1]:
import os
import glob
csvfiles = glob.glob("*.csv")
jsonfiles = glob.glob("*.json")
txtfiles = glob.glob("*.txt")
csvfiles

['fortune500.csv', 'merged.csv', 'Results_Combine.csv']

In [2]:
jsonfiles[0]

'lines.json'

In [3]:
txtfiles

['unstructureddata.txt']

In [4]:
os.listdir()

['.ipynb_checkpoints',
 'fortune500.csv',
 'Lab2_Max_Shuford.ipynb',
 'lines.json',
 'merged.csv',
 'Results_Combine.csv',
 'unstructureddata.txt']

> Lets look at the data before trying to clean it, we can skip bad data to see what we are expecting
> > We will look at all our available files

In [7]:
import pandas as pd
import numpy as np
filepath = 'fortune500.csv'
csvdf = pd.read_csv(filepath, on_bad_lines='skip')

In [11]:
import csv
a = 0
title = []
gooddata = []
baddata= []

with open(filepath, mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    for i, row in enumerate(csv_reader):
        if i == 0:                 # first line = headers
            title = row
            continue
        if len(row) == 5:          # keep only good rows
            gooddata.append(row)
        else:
            baddata.append(row)

title

['#Year',
 '(1)Rank',
 '!Company',
 '(3)Revenue (in millions)',
 'okjb)Profit (in millions)']

In [282]:
## clean our titles
import re

# strip edges
title = [t.lstrip('\ufeff').strip() for t in title]

# if there’s any ')' before the real name, drop everything up to and including that ')'
title = [re.sub(r'^[^)]*\)', '', t) for t in title]

# 3) now remove any leftover leading parentheses or spaces
title = [re.sub(r'^[()\s]+', '', t) for t in title]

# clean the rest and normalize spacing
title = [re.sub(r'[^A-Za-z()\s]+', '', t) for t in title]
title = [re.sub(r'\s+', ' ', t).strip() for t in title]

title

['Year', 'Rank', 'Company', 'Revenue (in millions)', 'Profit (in millions)']

In [283]:
baddata

[['1', '', '', 'l', '776.8', '22.7'],
 ['1955', '1'],
 ['1955', '', '', '', 'Du Mont (Allen B.) Labs.', '91.9', '0.9'],
 ['1955', '335', 'Eagle-Picher Industries', '', '', ''],
 ['', '', '', 'Rockwell Manufacturing', '76.5', '5.7'],
 ['1955', '412'],
 ['1956', '104', 'Crane', ''],
 ['19', '', '', ''],
 ['195', '', '', '6.9'],
 ['197', '', '', '.4'],
 ['1980', '186', 'Kopp2.4'],
 ['1980', '', '', ''],
 ['1980', '3', '', '', '1', '26.5'],
 ['1981', '57', 'Gulf & Western Industries', '', '', '']]

In [284]:
csvdf = pd.DataFrame(gooddata, columns=[str(c).strip() for c in title])
csvdf

,Year,Rank,Company,Revenue (in millions),Profit (in millions)
0,1955,1,General Motors,9823.5,806
1,1955,2,Exxon Mobil,5661.4,584.8
2,1955,3,U.S. Steel,3250.4,195.4
3,1955,4,General Electric,,212.6
4,1955,5,Esmark,2510.8,19.1
...,...,...,...,...,...
28838,2009,996,Tellabs,"1,729.00",-930.1
28839,2009,997,Administaff,"1,724.40",45.8
28840,2009,998,Sanderson Farms,"1,723.60",-43.1
28841,2009,999,MGIC Investment,"1,721.50",-518.9


In [285]:
## JSON TIME
import json

file_path = jsonfiles[0]  # Replace with your file path
data = []
title = ['Year','Rank','Company', 'Revenue (in millions)', 'Profit (in millions)']

with open(file_path, 'r') as file:
    for line in file:
        try:
            # Parse the JSON string
            json_obj = json.loads(line)

            # Handle missing data by setting a default value
            # Your code
            for key in title:
                if key not in json_obj:
                    json_obj[key] = None

            # Append the processed JSON object to the list
            data.append(json_obj)
        except json.JSONDecodeError:
            print(f'Invalid Json data: {line}')

jsondf = pd.DataFrame(data)
jsondf

Invalid Json data: {"Year":1977,"Rank":,"Company":"Shell Oil","Revenue (in millions)":"9230","Profit (in millions)":"705.8"}

Invalid Json data: {"Year":1977,"Rank":17,"Company":"Conoco","Revenue (in millions)":,"Profit (in millions)":"460"}

Invalid Json data: {"Year":1977?,"Rank":20,"Company":"Tenneco Automotive","Revenue (in millions)":"6389.2","Profit (in millions)":"383.5"}

Invalid Json data: {"Year":1977,"Rank":21,"Company":,"Revenue (in millions)":"6345.7","Profit (in millions)":"441.2"}

Invalid Json data: {"Year":1977,"Rank":26,"Company":"Occidental Petroleum","Revenue (in millions)":,"Profit (in millions)":"183.7"}

Invalid Json data: {"Year":1977,"Rank":43,"Company":,"Revenue (in millions)":"4086.8","Profit (in millions)":"136"}

Invalid Json data: {"Year":1977aka,"Rank":381,"Company":"National Service Industries","Revenue (in millions)":"482.7","Profit (in millions)":"24.9"}

Invalid Json data: {"Year":1977,"Rank":418,"Company":"Kellwood","Revenue (in millions)":,"Profit (

,Year,Rank,Company,Revenue (in millions),Profit (in millions)
0,1977,1,Exxon Mobil,48630.8,2641
1,1977,2,General Motors,47181,2902.8
2,1977,3,Ford Motor,28839.6,983.1
3,1977,4,Texaco,26451.9,869.7
4,1977,5,Mobil,26062.6,942.5
...,...,...,...,...,...
10869,1998,396,American Family Ins. Grp.,3689.4,251.6
10870,1998,397,Baker Hughes,3685.4,97
10871,1998,398,Service Merchandise,3662.8,-91.6
10872,1998,399,Silicon Graphics,3662.6,78.6


In [286]:
file_path = 'unstructureddata.txt'  # use your file path

## funct. to process a block of text and return a dict. 
def process_block(block):
    entity={}
    for item in block:
        key,value = item.split(':')
        entity[key] = value
    return entity

# Read and process the file
current_data= []
result= []
with open(file_path,'r') as file:
    for line in file:
        if line.strip() == "":
            ##print("1")
            dfcontent = process_block(current_data)
            result.append(dfcontent)
            current_data=[]
        else:
            ##print(line)
            current_data.append(line.strip())

##print(result)
txtdf = pd.DataFrame(result)
txtdf

,Year,Rank,Company,Revenue (in millions),Profit (in millions)
0,1998,401,Turner Corp.,3639.8,5.9
1,1998,402,Reebok International,3637.4,135.1
2,1998,403,Morton International,3636.5,343
3,1998,404,Engelhard,3630.7,47.8
4,1998,405,Temple-Inland,3625.4,50.8
...,...,...,...,...,...
7595,2009,996,Tellabs,"1,729.00",-930.1
7596,2009,997,Administaff,"1,724.40",45.8
7597,2009,998,Sanderson Farms,"1,723.60",-43.1
7598,2009,999,MGIC Investment,"1,721.50",-518.9


In [287]:
merged = pd.concat([csvdf, jsondf, txtdf], ignore_index=True, sort=False)

In [288]:
merged

,Year,Rank,Company,Revenue (in millions),Profit (in millions)
0,1955,1,General Motors,9823.5,806
1,1955,2,Exxon Mobil,5661.4,584.8
2,1955,3,U.S. Steel,3250.4,195.4
3,1955,4,General Electric,,212.6
4,1955,5,Esmark,2510.8,19.1
...,...,...,...,...,...
47312,2009,996,Tellabs,"1,729.00",-930.1
47313,2009,997,Administaff,"1,724.40",45.8
47314,2009,998,Sanderson Farms,"1,723.60",-43.1
47315,2009,999,MGIC Investment,"1,721.50",-518.9


In [289]:
merged.to_csv("merged.csv", index=False)

# 2. Refinement

In [290]:
import re
import pandas as pd

df = merged.copy()

# Normalize column names: lower, underscores, no weird chars
df.columns = (df.columns.str.strip()
                         .str.lower()
                         .str.replace(r'[^a-z0-9_]+', '_', regex=True)
                         .str.replace(r'(^_|_$)', '', regex=True))

# Clean company name if present
if 'company' in df.columns:
    df['company'] = df['company'].astype(str).str.strip()

# Identify columns for 1995–1998 
year_pat = r'1995|1996|1997|1998'
num_cols = [c for c in df.columns if (('revenue' in c or 'profit' in c or re.search(year_pat, c)))]

# Coerce columns
for c in num_cols:
    if c in df.columns:
        df[c] = (df[c].astype(str)
                      .str.replace(r'[^\d\-\.\(\)]', '', regex=True)      # drop $, commas, spaces, etc.
                      .str.replace(r'\(([\d\.]+)\)', r'-\1', regex=True)) # (123) -> -123
        df[c] = pd.to_numeric(df[c], errors='coerce')

df_clean = df


In [291]:
df

,year,rank,company,revenue_in_millions,profit_in_millions
0,1955,1,General Motors,9823.5,806.0
1,1955,2,Exxon Mobil,5661.4,584.8
2,1955,3,U.S. Steel,3250.4,195.4
3,1955,4,General Electric,NaN,212.6
4,1955,5,Esmark,2510.8,19.1
...,...,...,...,...,...
47312,2009,996,Tellabs,1729.0,-930.1
47313,2009,997,Administaff,1724.4,45.8
47314,2009,998,Sanderson Farms,1723.6,-43.1
47315,2009,999,MGIC Investment,1721.5,-518.9


> Lets refine further with our merged dataset

In [292]:
years = ["1995","1996","1997","1998"]

# keep only rows with a company name
good = good[good["company"].notna()].copy()

# filter the target years 
mask = good["year"].astype(str).str.strip().isin(years)
g = good.loc[mask, ["company", "year", "revenue_in_millions", "profit_in_millions"]].copy()

# coerce revenue and profit to numeric
for c in ["revenue_in_millions", "profit_in_millions"]:
    g[c] = (g[c].astype(str)
                           .str.replace(r'[^\d\-\.\(\)]', '', regex=True)
                           .str.replace(r'\(([\d\.]+)\)', r'-\1', regex=True))
    g[c] = pd.to_numeric(g[c], errors="coerce")

# for my sanity
g.head()


,company,year,revenue_in_millions,profit_in_millions
19776,General Motors,1995,154951.2,4900.6
19777,Ford Motor,1995,128439.0,5308.0
19778,Exxon Mobil,1995,101459.0,5100.0
19779,Wal-Mart Stores,1995,83412.4,2681.0
19780,AT&T,1995,75094.0,4676.0


In [293]:
## Calculate our totals

rev_totals = (g.groupby("company", dropna=True)["revenue_in_millions"]
                 .sum(min_count=1))
profit_totals = (g.groupby("company", dropna=True)["profit_in_millions"]
                    .sum(min_count=1))

top_rev_company    = rev_totals.dropna().idxmax() if not rev_totals.dropna().empty else pd.NA
top_rev_value_m    = rev_totals.dropna().max()    if not rev_totals.dropna().empty else pd.NA
top_profit_company = profit_totals.dropna().idxmax() if not profit_totals.dropna().empty else pd.NA
top_profit_value_m = profit_totals.dropna().max()    if not profit_totals.dropna().empty else pd.NA

# quick check
rev_totals.sort_values(ascending=False).head(10)
rcdf = pd.DataFrame(rev_totals)
rcdf

,revenue_in_millions
company,
3Com,6294.2
3M,120980.0
AFLAC,48204.4
AGCO,6448.8
AMP,30240.6
...,...
Wyeth,101253.2
Xcel Energy,18317.4
Xerox,148974.0


# 3. Save our data

In [294]:
# freak out if this doesnt work
Results_Combine.loc[Results_Combine["Metric"]=="Top revenue 1995-1998 (company)", "Value"] = top_rev_company
Results_Combine.loc[Results_Combine["Metric"]=="Top revenue 1995-1998 (sum)",     "Value"] = top_rev_value_m
Results_Combine.loc[Results_Combine["Metric"]=="Top profit 1995-1998 (company)",  "Value"] = top_profit_company
Results_Combine.loc[Results_Combine["Metric"]=="Top profit 1995-1998 (sum)",      "Value"] = top_profit_value_m

Results_Combine

,Metric,Value
0,Good data rows,46623
1,Bad data rows,694
2,Unique companies,2574
3,Top revenue 1995-1998 (company),General Motors
4,Top revenue 1995-1998 (sum),1172276.6
5,Top profit 1995-1998 (company),Exxon Mobil
6,Top profit 1995-1998 (sum),47570.0


In [295]:
## save our finalized data to our csv
Results_Combine.to_csv("Results_Combine.csv", index=False)